<a href="https://colab.research.google.com/github/woncoh1/sas2csv/blob/main/sas2csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructions
- Purpose
    - This notebook converts `.sas7bdat` files, stored in Google Drive, to `.csv` files
    - The raw `.sas7bdat` files can be moved to trash or left intact
- File permission
    - You need write (editor) permission for the folder of interest (project folder) and its parent folder
    - Only owners can move exisiting files to trash
- Folder structure
    - Project Folder
        - Sub-folder 1
            - `.sas7bdat` 1
            - `.sas7bdat` 2
            - ...
            - `.sas7bdat` n
        - Sub-folder 2
        - ...
        - Sub-folder n

# Import Libraries

In [ ]:
library(tidyverse)
library(haven)
library(googledrive)

# Define Constants

In [ ]:
SAS_ENCODING = 'cp949'
IPYTHON_PATH = '/usr/local/lib/python3.7/dist-packages/google/colab/_ipython.py'

In [ ]:
#@title Select Project! { run: "auto" }
PROJECT = 'KoGES' #@param ['KNHANES', 'KoGES']
PROJECTS = c(
    KNHANES='1-IvmZpUepUMHglmhjOL5MtJ1NknySY30',
    KoGES='1hW4UXbINgLsO_O8i1abHEcsjXPT1A8q4'
)
DATA <- PROJECTS[[PROJECT]]

In [ ]:
#@title Delete `.sas7bdat`? { run: "auto" }
DELETE = 'No' #@param ['Yes', 'No']
DELETES = c(
    No=FALSE,
    Yes=TRUE
)
DELETE_RAW <- DELETES[[DELETE]]

# Authenticate Drive
- In order to access files in Google Drive, we need authentication, which requires an interactive R session
- Unfortunately, R session in Colab is non-interactive by default, so we need to monkey-patch, as described [here](https://towardsdatascience.com/how-to-use-r-in-google-colab-b6e02d736497#2176):

In [ ]:
if (file.exists(IPYTHON_PATH)) {
    install.packages('R.utils')
    library('R.utils')
    library('httr')
    reassignInPackage('is_interactive', pkgName='httr', function() TRUE)
    options(rlang_interactive=TRUE)
}

In [ ]:
drive_auth(use_oob=TRUE, cache=TRUE)

# List Folders

In [ ]:
with_drive_quiet(
    folders <- DATA %>%
        as_id() %>%
        as_dribble() %>%
        drive_ls(type='folder', trashed=FALSE) %>%
        arrange(name)
)
folders %>% select(id, name)

# Convert Files

In [ ]:
for (folder in seq_along(folders$id)) {
    with_drive_quiet(
        files <- folders[folder,]$id %>%
            as_id() %>%
            as_dribble() %>%
            drive_ls(q="fileExtension contains 'sas7bdat'", trashed=FALSE) %>%
            arrange(name)
    )
    for (file in seq_along(files$id)) {
        sas <- files[file,]$name
        csv <- sas %>% str_replace('\\..*', '.csv')
        raw <- files[file,]$id
        parent <- files[file,]$drive_resource[[1]]$parents[[1]]
        raw %>% drive_download(overwrite=TRUE)
        sas %>% read_sas(encoding=SAS_ENCODING) %>% write_excel_csv(csv)
        csv %>% drive_upload(path=as_id(parent), overwrite=TRUE)
        if (DELETE_RAW) raw %>% as_dribble() %>% drive_trash()
        paste('rm', sas) %>% system(intern=TRUE)
        paste('rm', csv) %>% system(intern=TRUE)
        cat(paste(round(file/nrow(files)*100, digits=2), '% files\n'))
        cat(paste(round(folder/nrow(folders)*100, digits=2), '% folders\n\n'))
    }
}

# References
1. https://developers.google.com/drive/api/v2/reference/files/trash
2. https://developers.google.com/drive/api/guides/ref-roles